In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install flair

     |████████████████████████████████| 286kB 17.9MB/s 
     |████████████████████████████████| 71kB 7.0MB/s 
     |████████████████████████████████| 776.8MB 12kB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 983kB 49.2MB/s 
     |████████████████████████████████| 798kB 48.6MB/s 
     |████████████████████████████████| 1.2MB 47.9MB/s 
     |████████████████████████████████| 19.7MB 163kB/s 
     |████████████████████████████████| 1.9MB 36.3MB/s 
     |████████████████████████████████| 3.2MB 45.0MB/s 
     |████████████████████████████████| 890kB 52.0MB/s 
  Created wheel for gdown: filename=gdown-3.12.2-cp37-none-any.whl size=9693 sha256=9095db2764970439f97fdc86ec0831190c175236db82139da3f6b76a2d71455f
  Stored in directory: /root/.cache/pip/wheels/81/d0/d7/d9983facc6f2775411803e0e2d30ebf98efbf2fc6e57701e09
Successfully built gdown
  Created wheel for sqlitedict: fil

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow_hub as hub

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from textblob import TextBlob
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from flair.models import TextClassifier
from flair.data import Sentence

/usr/local/lib/python3.7/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [ ]:
DATASET_ENCODING = "ISO-8859-1"

In [ ]:
colnames = ['target','ids','date','flag','user','text']
df = pd.read_csv('/content/drive/MyDrive/FA PROJECT 2021/kaggle_tweets.csv', names = colnames, header = None, encoding = DATASET_ENCODING)
df.head()

,target,ids,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [ ]:
#drop irrelevant columns
df = df.drop(columns=['ids','date','flag','user'])
df.head()

,target,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


In [ ]:
#check no missing data
df.isnull().values.any()

False

In [ ]:
#target: the polarity of the tweet (0 = negative, 2 = neutral, 4 = positive)
# remove neutral rows
df = df[df["target"] != 2]

In [ ]:
# change target 4 to 1 --> so that 0 means negative, 1 means positive
df['target'] = df['target'].replace(4, 1)

In [ ]:
df

,target,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."
...,...,...
1599995,1,Just woke up. Having no school is the best fee...
1599996,1,TheWDB.com - Very cool to hear old Walt interv...
1599997,1,Are you ready for your MoJo Makeover? Ask me f...
1599998,1,Happy 38th Birthday to my boo of alll time!!! ...


In [ ]:
!pip install emoji

     |████████████████████████████████| 133kB 16.3MB/s 


In [ ]:
import re
import emoji

def cleaner(tweet):
    tweet = tweet.lower()
    if tweet[:2] == 'rt':
        tweet = re.sub(r'^.*?:', ':', tweet)
    tweet = re.sub("@[A-Za-z0-9]+","",tweet) #Remove @ sign
    tweet = re.sub(r"(?:\@|http?\://|https?\://|www)\S+", "", tweet) #Remove http links
    tweet = " ".join(tweet.split())
    tweet = ''.join(c for c in tweet if c not in emoji.UNICODE_EMOJI) #Remove Emojis
    tweet = tweet.replace("#", "").replace("_", " ")
    tweet = tweet.replace("RT ", "")
    tweet = re.sub(r'[^\w\s]','',tweet) #remove punctuation
    tweet = re.sub(r"(^|\W)\d+", "", tweet)# remove digits
    return tweet
df['processed_tweet'] = df['text'].map(lambda x: cleaner(x))

In [ ]:
df.head()

,target,text,processed_tweet
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",a thats a bummer you shoulda got david carr o...
1,0,is upset that he can't update his Facebook by ...,is upset that he cant update his facebook by t...
2,0,@Kenichan I dived many times for the ball. Man...,i dived many times for the ball managed to sav...
3,0,my whole body feels itchy and like its on fire,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all....",no its not behaving at all im mad why am i her...


In [ ]:
#split into train and test
train_df,test_df = train_test_split(df,test_size=0.2)

In [ ]:
train_df

,target,text,processed_tweet
162667,0,I hope I get the job so I can have some steady...,i hope i get the job so i can have some steady...
1473936,1,Listenin 2 beat again !!! x,listenin beat again x
1207393,1,i love midnight talks with debbie she's the b...,i love midnight talks with debbie shes the bes...
852787,1,Im gonna see Raphael Saadiq perform live tonight,im gonna see raphael saadiq perform live tonight
1281482,1,@djsamyoung send me some beats !! Also fotos f...,send me some beats also fotos from b8
...,...,...,...
1270522,1,@wanesaleto thanks how are you today?,thanks how are you today
1465876,1,The sun is shining &amp; so am I.. Bout to sta...,the sun is shining amp so am i bout to start m...
903769,1,"like spirits in the night ooooh night, quï¿½ g...",like spirits in the night ooooh night quï½ gra...
715399,0,"bloody key card system took a shit,people cant...",bloody key card system took a shitpeople cant ...


In [ ]:
test_df

,target,text,processed_tweet
603899,0,Hmm coffee shop girls appear to have gone off ...,hmm coffee shop girls appear to have gone off me
821818,1,I feel better now,i feel better now
354733,0,watching tennis.....I missed so much,watching tennisi missed so much
901417,1,"If anyone needs help with images, let me know ...",if anyone needs help with images let me know a...
917643,1,LOVE ME LOVE ME SAY THAT YOU LOVE ME...,love me love me say that you love me
...,...,...,...
287578,0,"@MelissaLoschy Awesome! Yea, Its going to be r...",awesome yea its going to be really good i was ...
103981,0,I need to get away from here for awhile. Even ...,i need to get away from here for awhile even i...
285181,0,had to cancel the beach trip today. it's supp...,had to cancel the beach trip today its suppose...
1401609,1,@Rocks4Ever bet u say that 2all the girls Wot...,bet u say thatall the girls wots this gas comi...


## Textblob

In [ ]:
def text_sentiment(text):
    testimonial = TextBlob(text)
    return int(testimonial.sentiment.polarity>0.5)

predictions = test_df.text.map(lambda x :  text_sentiment(x))

In [ ]:
accuracy_score(predictions,test_df.target)

0.562971875

In [ ]:
!pip install nltk

## Vader

In [ ]:
import nltk
nltk.downloader.download('vader_lexicon')

analyzer = SentimentIntensityAnalyzer()

def text_sentiment_vader(text):
    vs = analyzer.polarity_scores(text)
    return int(vs.get("compound")>0)
 
predictions = test_df.text.map(lambda x : text_sentiment_vader(x))

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


In [ ]:
accuracy_score(predictions,test_df.target)

0.6492625

## Flair

In [ ]:
classifier = TextClassifier.load('en-sentiment')

def text_sentiment_flair(text):
    sentence = Sentence(text)
    classifier.predict(sentence)
    return np.round(sentence.labels[0].score)

predictions = test_df.text.map(lambda x : text_sentiment_flair(x))

2021-03-11 03:22:22,840 https://nlp.informatik.hu-berlin.de/resources/models/sentiment-curated-distilbert/sentiment-en-mix-distillbert_4.pt not found in cache, downloading to /tmp/tmpn26ctjel


100%|██████████| 265512723/265512723 [00:09<00:00, 27335798.48B/s]

2021-03-11 03:22:32,868 copying /tmp/tmpn26ctjel to cache at /root/.flair/models/sentiment-en-mix-distillbert_4.pt


2021-03-11 03:22:33,430 removing temp file /tmp/tmpn26ctjel
2021-03-11 03:22:33,458 loading file /root/.flair/models/sentiment-en-mix-distillbert_4.pt


In [ ]:
accuracy_score(predictions,test_df.target)

0.4992